In [2]:
import os

kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

import shutil
shutil.copy("kaggle.json", os.path.join(kaggle_dir, "kaggle.json"))

os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

In [3]:
import kaggle
kaggle.api.authenticate()

DATA_ROOT_DIR_PATH = os.path.join('data', 'fer2013')

# Check if dataset already exists before downloading
if os.path.exists('data') and os.path.exists(DATA_ROOT_DIR_PATH):
    print("Dataset already exists, skipping download")
else:
    kaggle.api.dataset_download_files('msambare/fer2013', path=DATA_ROOT_DIR_PATH, unzip=True)
    print("Dataset downloaded")

Dataset already exists, skipping download


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
# from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import glob

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
TRAIN_IMAGE_DIR_PATH = os.path.join(DATA_ROOT_DIR_PATH, "train")
TEST_IMAGE_DIR_PATH = os.path.join(DATA_ROOT_DIR_PATH, "test")
BATCH_SIZE = 128
IMAGE_SIZE = 48
LR = 0.001
EPOCHS = 10
NUM_WORKERS = 4

In [7]:
dirs = os.listdir(TRAIN_IMAGE_DIR_PATH)
labels_map = {label: index for index, label in enumerate(dirs)}
labels_map

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [16]:
train_data: list[tuple[str, int]] = []
test_data: list[tuple[str, int]] = []

for label, index in labels_map.items():
    train_image_paths = [(image_path, index) for image_path in glob.glob(f"{TRAIN_IMAGE_DIR_PATH}/{label}/*.jpg")]
    test_image_paths = [(image_path, index) for image_path in glob.glob(f"{TEST_IMAGE_DIR_PATH}/{label}/*.jpg")]
    
    train_data.extend(train_image_paths)
    test_data.extend(test_image_paths)

train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=420, stratify=[label for _, label in train_data])

In [17]:
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Grayscale(1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5], std = [0.5]),
])

test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5], std = [0.5]),
])

In [18]:
class EmotionDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image_path, label = self.data[index]
        image = Image.open(image_path).convert('L')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

In [19]:
train_dataset = EmotionDataset(train_data, train_transform)
val_dataset = EmotionDataset(val_data, test_transform)
test_dataset = EmotionDataset(test_data, test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [20]:
class EmotionsDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 3 * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 7),
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [21]:
model = EmotionsDetector().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [22]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, device, epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
        # Training
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
        train_losses.append(running_loss / total)
        train_accuracies.append(correct / total)

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)
        val_losses.append(val_loss / val_total)
        val_accuracies.append(val_correct / val_total)

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_losses[-1]:.4f}, Acc: {train_accuracies[-1]:.4f} | "
              f"Val Loss: {val_losses[-1]:.4f}, Acc: {val_accuracies[-1]:.4f}")

    return train_losses, val_losses, train_accuracies, val_accuracies

In [23]:
def test_model(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy, all_preds, all_labels

In [26]:
def plot_metrics(train_losses, val_losses, train_acc, val_acc):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize = (12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label = 'Train Loss')
    plt.plot(epochs, val_losses, label = 'Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curve')

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, label = 'Train Acc')
    plt.plot(epochs, val_acc, label = 'Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy Curve')
    plt.show()

In [ ]:
train_losses, val_losses, train_acc, val_acc = train_and_validate(
    model, train_loader, val_loader, criterion, optimizer, device, EPOCHS
)

In [ ]:
plot_metrics(train_losses, val_losses, train_acc, val_acc)

In [ ]:
test_acc, test_preds, test_labels = test_model(model, test_loader, device)
print(f"Accuracy = {test_acc}")
print(f"Test Predictions = {test_preds}")
print(f"Test Labels = {test_labels}")